In [7]:
from openai import OpenAI
import json
from together import Together

client = OpenAI(
    # base_url="https://kolank.com/api/v1",
    # base_url = "https://api.deepseek.com",
    # base_url = "https://api.together.xyz/v1/",
    # api_key="sk-..."
    # api_key = "7ea...3fb"
    # api_key="dc1...0ee" 
)

file_path = './output.json'

system_prompt = """
Extract the following fields from OCR results of Persian (right-to-left) documents and return a JSON object. Ensure the output adheres to the requirements below.

Fields to Extract:

name (نام و نام خانوادگی): Full name of the individual, do not include father name. name and last name could include compound names.

national_id: National identification number.

organization_name: Name of the organization.

net_payment (خالص دریافتی یا قابل پرداخت و...): Net payment or net payable amount.

If numbers are written in words, verify them against the detected numerical value.

If the numerical value and its written equivalent do not match, prioritize the written equivalent if it is consistent across OCR engines and matches the context. Otherwise, prioritize the numerical value.

year_and_month (هجری شمسی): Year and month in the Persian (Shamsi) calendar format.

Rules:

If a field has no value, return an empty string ("") for that field.

Include a confidence field (0 to 100) representing the confidence level of the extracted data.

Do not include any additional text, explanations, or titles in the output—only the JSON object.

For net_payment:

Carefully cross-verify numerical values with their written equivalents.

Prioritize the written equivalent if it is consistent across OCR engines and matches the context of "خالص دریافتی" or similar phrases.

If the written equivalent is missing or inconsistent, prioritize the numerical value. sometimes written text is separated in multiple blocks, you should move some part of it to fit

Ensure the output is accurate and adheres to the rules strictly.

Output Format:

```json
{
  "name": "",
  "national_id": "",
  "organization_name": "",
  "net_payment": "",
  "year_and_month": "",
  "confidence": 0
}
```
Additional Notes:

The documents are in Persian and written right-to-left.
The confidence field should reflect the overall confidence in the accuracy of the extracted data.

Return only the JSON object as the final output.

"""

with open(file_path, 'r') as file:
    data = json.load(file)  

# json_result = data["16a13165-e5cb-47e1-9efd-5794c6c41a89-1403_10_11.jpeg"]
json_result = data["30034b7a-7a44-4438-bc11-874d5cc40377-1403_10_11.jpeg"]
results = {}

def together(texts, model):
    
    client = Together()
    
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=[
        {
            "role": "system", 
            "content": system_prompt}, 
        {
            "role": "user", 
            "content": "Here is extracted ocr texts: " + json.dumps(texts)
        }
    ],
        max_tokens=null,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<｜end▁of▁sentence｜>"],
        stream=True
    )
    for token in response:
        if hasattr(token, 'choices'):
            print(token.choices[0].delta.content, end='', flush=True)

    

def process(texts, model):
    result = client.chat.completions.create(model=model, messages = [
        {
            "role": "system", 
            "content": system_prompt}, 
        {
            "role": "user", 
            "content": "Here is extracted ocr texts: " + json.dumps(texts)
        }
    ], 
                           max_tokens=220
                                            # response_format={"type": "json_object"}
   )
    
    return result.choices[0].message.content

def process_file(texts): 
    deepseek = together(texts, "deepseek-ai/deepseek-llm-67b-chat")
    print("deepseek")
    deepseekv3 = together(texts, "deepseek-ai/DeepSeek-V3")
    print("deepseekv3")
    # llama = process(texts, "Meta-llama/llama-3.1-405b-instruct")
    # print("llama")
    # gemini = process(texts, "google/gemini-pro")
    # print("gemini")
    # databricks = process(texts, "Databricks/dbrx-instruct")
    # print("databricks")
    # qwen = process(texts, "Qwen/Qwen1.5-0.5B-Chat")
    # print("qwen")
    # grok = process(texts, "x-ai/grok-beta")
    # print("grok")

    result = { 
        "deepseek": deepseek, 
        "deepseekv3": deepseekv3, 
        # "llama": llama, 
        # "gemini": gemini, 
        # "databricks": databricks, 
        # "qwen": qwen, 
        # "grok": grok
    }
    
    return result

# print(process_file(data['4bb7c380-f9c4-4301-a5da-faf015a2ab6d-1403_10_11.jpeg']))


with open("deepseek.json", "w") as file:
    for key in data:
        print(key)
        results[key] = process_file(data[key])    
        json.dump(results, file, ensure_ascii=False, indent=4)
        
print("DONE")


ImportError: cannot import name 'can_be_positional' from 'pydantic._internal._utils' (/mnt/Users/UT/Desktop/anaconda4/lib/python3.12/site-packages/pydantic/_internal/_utils.py)

In [ ]:
api_key="sk-bb292a0b223946fe86be4dd0cd7f9097"
